# Hugging Face: Salesforce/blip-vqa-capfilt-large

In [26]:
import torch
import requests
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

In [ ]:
model_name = "microsoft/git-large-textvqa"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [39]:
def is_product_available(question: str, img_url: str):
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
    pixel_values = processor(images=raw_image, return_tensors="pt").pixel_values

    input_ids = processor(text=question, add_special_tokens=False).input_ids
    input_ids = [processor.tokenizer.cls_token_id] + input_ids
    input_ids = torch.tensor(input_ids).unsqueeze(0)

    generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)
    
    return processor.batch_decode(generated_ids, skip_special_tokens=True)

retailers = [
    {
        "name": "alkosto.com",
        "prompt": "Is the product sold out?",
        "in_stock": "https://img001.prntscr.com/file/img001/Rd5e6SJARZm9OplHevI54A.png",
        "out_of_stock": "https://img001.prntscr.com/file/img001/v3JQkBjSQ2KzmNDFJf8DfA.png"
    },
    {
        "name": "alza.cz",
        "prompt": "Is the price available for the product?",
        "in_stock": "https://img001.prntscr.com/file/img001/RXWow4UKTDKXySEshfV6Cg.png",
        "out_of_stock": "https://img001.prntscr.com/file/img001/6aWiJaT8Q5SipHDyhM20JA.png"
    }
]

for retailer in retailers:
    print(f"{retailer['name']}")
    is_available = is_product_available(retailer["prompt"], retailer["in_stock"])
    print(f"In-Stock: {is_available}")
    
    is_available = is_product_available(retailer["prompt"], retailer["out_of_stock"])
    print(f"Out-of-Stock: {is_available}")
    
    print("-----------------------------------\n")

alkosto.com
In-Stock: ['is the product sold out? yes']
Out-of-Stock: ['is the product sold out? yes']
-----------------------------------
